# Amazon Bedrock AgentCore를 활용한 External Database Agent 배포

이 노트북에서는 `01_external_dbs.ipynb`의 MCP 기반 외부 데이터베이스 연동을 Amazon Bedrock AgentCore Runtime으로 배포합니다.

## 학습 목표
- MCP 서버를 AgentCore Runtime에 통합
- Strands Agent를 Production 환경에 배포
- 외부 데이터베이스 연동 Agent를 서버리스로 운영

## 1. 환경 설정

In [ ]:
# 필요한 패키지 설치
%pip install bedrock-agentcore-starter-toolkit strands-agents strands-agents-tools boto3 mcp arxiv chembl-webresource-client python-dateutil pubmedmcp --quiet

In [ ]:
# 라이브러리 임포트
import sys
import json
import logging
from typing import Dict, Any

# AWS SDK
import boto3
from boto3.session import Session

# Bedrock AgentCore
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Strands Agents
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 2. MCP 서버 Agent 구현

외부 데이터베이스를 연동하는 통합 Agent를 구현합니다.

In [ ]:
%%writefile external_db_agent.py
"""
External Database Agent for Amazon Bedrock AgentCore
Arxiv, ChEMBL, PubMed, ClinicalTrials 통합
"""
import json
import logging
from typing import Dict, Any
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent, tool
from strands.models import BedrockModel

# 외부 데이터베이스 도구들
import arxiv
from chembl_webresource_client.new_client import new_client as chembl_client
import httpx
from defusedxml import ElementTree as ET

app = BedrockAgentCoreApp()
logger = logging.getLogger(__name__)

# Arxiv 도구
@tool
def search_arxiv(query: str, max_results: int = 5):
    """Arxiv에서 논문을 검색합니다"""
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    
    results = []
    for paper in client.results(search):
        results.append({
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "abstract": paper.summary[:500],
            "url": paper.pdf_url
        })
    return results

# ChEMBL 도구
@tool
def search_compound(compound_name: str):
    """ChEMBL에서 화합물 정보를 검색합니다"""
    try:
        molecule = chembl_client.molecule.filter(
            pref_name__iexact=compound_name
        ).only(['molecule_chembl_id', 'pref_name', 'max_phase'])
        
        if molecule:
            mol_data = molecule[0]
            # IC50 활성 데이터 가져오기
            activity = chembl_client.activity.filter(
                molecule_chembl_id=mol_data['molecule_chembl_id']
            ).filter(standard_type="IC50").only(
                ['pchembl_value', 'assay_description']
            )[:10]
            
            return {
                "chembl_id": mol_data['molecule_chembl_id'],
                "name": mol_data['pref_name'],
                "max_phase": mol_data.get('max_phase'),
                "activities": list(activity)
            }
    except Exception as e:
        return {"error": str(e)}

# PubMed 도구  
@tool
def search_pubmed(query: str, max_results: int = 5):
    """PubMed에서 의학 문헌을 검색합니다"""
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils"
    
    # 검색
    search_params = {
        "db": "pubmed",
        "term": query,
        "retmax": max_results,
        "retmode": "json"
    }
    
    search_response = httpx.get(
        f"{base_url}/esearch.fcgi",
        params=search_params
    )
    search_data = search_response.json()
    
    id_list = search_data["esearchresult"].get("idlist", [])
    if not id_list:
        return []
    
    # 상세 정보 가져오기
    fetch_params = {
        "db": "pubmed",
        "id": ",".join(id_list),
        "retmode": "xml"
    }
    
    fetch_response = httpx.get(
        f"{base_url}/efetch.fcgi",
        params=fetch_params
    )
    
    # XML 파싱
    root = ET.fromstring(fetch_response.text)
    articles = []
    
    for article in root.findall(".//PubmedArticle"):
        title = article.find(".//ArticleTitle")
        abstract = article.find(".//AbstractText")
        pmid = article.find(".//PMID")
        
        articles.append({
            "pmid": pmid.text if pmid is not None else "",
            "title": title.text if title is not None else "",
            "abstract": abstract.text[:500] if abstract is not None else ""
        })
    
    return articles

# 모델 설정
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(model_id=model_id)

# Agent 생성
agent = Agent(
    model=model,
    tools=[search_arxiv, search_compound, search_pubmed],
    system_prompt="""당신은 생명과학 연구 어시스턴트입니다.
    Arxiv, ChEMBL, PubMed 데이터베이스를 활용하여 포괄적인 연구 정보를 제공합니다.
    각 데이터베이스의 정보를 종합하여 과학적이고 정확한 답변을 제공하세요."""
)

@app.entrypoint
def external_db_agent(payload: Dict[str, Any]) -> str:
    """AgentCore 엔트리포인트"""
    user_input = payload.get("prompt", "")
    logger.info(f"Processing request: {user_input}")
    
    try:
        response = agent(user_input)
        return response.message['content'][0]['text']
    except Exception as e:
        logger.error(f"Error: {str(e)}")
        return f"오류가 발생했습니다: {str(e)}"

if __name__ == "__main__":
    app.run()

## 3. Requirements 파일 생성

In [ ]:
%%writefile requirements.txt
strands-agents>=0.5.0
strands-agents-tools>=0.1.0
boto3>=1.34.0
mcp>=0.1.0
arxiv>=2.1.0
chembl-webresource-client>=0.10.8
python-dateutil>=2.8.2
httpx>=0.24.0
defusedxml>=0.7.1

## 4. AgentCore Runtime 구성 및 배포

In [ ]:
# AWS 세션 설정
boto_session = Session()
region = boto_session.region_name

# Runtime 인스턴스 생성
agentcore_runtime = Runtime()
agent_name = "external_db_research_agent"

# AgentCore Runtime 구성
response = agentcore_runtime.configure(
    entrypoint="external_db_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)

print("✅ AgentCore Runtime 구성 완료:")
response

## 5. Agent 배포

In [ ]:
# AgentCore Runtime에 배포
print("🚀 Agent를 AgentCore Runtime에 배포 중...")
launch_result = agentcore_runtime.launch()

print(f"\n✅ 배포 완료!")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"ECR URI: {launch_result.ecr_uri}")

## 6. 배포 상태 확인

In [ ]:
import time

# 배포 상태 확인
print("⏳ Agent 상태 확인 중...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(f"상태: {status}")

print(f"\n✅ 최종 상태: {status}")

## 7. Agent 테스트

In [ ]:
# Agent 호출 테스트
test_queries = [
    "COVID-19 백신 개발에 대한 최신 연구를 조사해주세요",
    "Aspirin의 화학적 특성과 의학적 용도를 알려주세요",
    "CRISPR 유전자 편집 기술의 최근 발전사항을 요약해주세요"
]

for query in test_queries:
    print(f"\n🔬 질문: {query}")
    print("=" * 60)
    
    invoke_response = agentcore_runtime.invoke({"prompt": query})
    
    if 'response' in invoke_response:
        response_text = invoke_response['response'][0]
        print(f"📊 응답:\n{response_text}")
    else:
        print("응답 없음")
    
    print("=" * 60)

## 8. Boto3를 통한 Agent 호출

In [ ]:
# Boto3 클라이언트로 Agent 호출
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

# 질문 예시
query = "알츠하이머병 치료제 개발 현황을 알려주세요"

print(f"🔬 질문: {query}")
print("=" * 60)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=launch_result.agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": query})
)

# 응답 처리
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                content.append(line[6:])
    
    print(f"📊 응답:\n{' '.join(content)}")
else:
    events = []
    for event in boto3_response.get("response", []):
        events.append(event)
    if events:
        print(f"📊 응답:\n{json.loads(events[0].decode('utf-8'))}")

## 9. 모니터링 및 로깅

In [ ]:
# CloudWatch 메트릭 확인
cloudwatch = boto3.client('cloudwatch', region_name=region)

# Agent 호출 메트릭 조회
from datetime import datetime, timedelta

end_time = datetime.utcnow()
start_time = end_time - timedelta(hours=1)

try:
    metrics = cloudwatch.get_metric_statistics(
        Namespace='AWS/BedrockAgentCore',
        MetricName='InvocationCount',
        Dimensions=[
            {'Name': 'AgentName', 'Value': agent_name}
        ],
        StartTime=start_time,
        EndTime=end_time,
        Period=300,
        Statistics=['Sum']
    )
    
    print("📊 최근 1시간 호출 통계:")
    for datapoint in metrics.get('Datapoints', []):
        print(f"  시간: {datapoint['Timestamp']}, 호출 수: {datapoint['Sum']}")
except Exception as e:
    print(f"메트릭 조회 실패: {e}")

## 10. 정리 (Optional)

In [ ]:
# Agent Runtime 삭제
# 주의: 이 셀을 실행하면 배포된 Agent가 삭제됩니다!

# agentcore_control_client = boto3.client(
#     'bedrock-agentcore-control',
#     region_name=region
# )

# ecr_client = boto3.client('ecr', region_name=region)

# # Runtime 삭제
# runtime_delete_response = agentcore_control_client.delete_agent_runtime(
#     agentRuntimeId=launch_result.agent_id
# )

# # ECR 리포지토리 삭제
# ecr_response = ecr_client.delete_repository(
#     repositoryName=launch_result.ecr_uri.split('/')[1],
#     force=True
# )

# print("✅ 정리 완료")

## 요약

이 노트북에서는 외부 데이터베이스(Arxiv, ChEMBL, PubMed)를 연동하는 Strands Agent를 Amazon Bedrock AgentCore Runtime에 성공적으로 배포했습니다.

### 주요 성과:
- ✅ MCP 서버 통합 Agent 구현
- ✅ AgentCore Runtime 자동 구성 및 배포
- ✅ 서버리스 환경에서 Agent 운영
- ✅ CloudWatch를 통한 모니터링

### 다음 단계:
1. 추가 데이터베이스 통합 (ClinicalTrials.gov 등)
2. Agent 응답 캐싱 구현
3. 비동기 처리 최적화
4. 사용자 인증 및 권한 관리 추가